## Imports

In [1]:
import pandas as pd

from sklearn.pipeline import Pipeline

import spacy
spacy_lemmatizer = spacy.load('en', disable=['parser', 'ner'])

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn import preprocessing

import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

from pprint import pprint

from timeit import default_timer

## Utilities

In [2]:
def lemmatize_spacy(text):
    """Apply Lemmatization using Spacy Lemmatization"""
    lemma_text = None
    if isinstance(text, str):
        lemma_text = ""

        # Parse the sentence using the loaded 'en' model object `nlp`
        doc = spacy_lemmatizer(text)

        # Extract the lemma for each token
        lemmas = []
        for token in doc:
            lemma = token.lemma_
            if lemma == '-PRON-': #https://spacy.io/api/annotation#lemmatization
                lemma = token.text
            if lemma.isalpha():
                lemmas.append(lemma)
        lemma_text = " ".join(lemmas)
        if len(lemma_text) == 0:
            return None
    return lemma_text

In [3]:
def convert_sec(no_of_secs):
    """return no_of_secs to min or hrs string"""
    if no_of_secs < 60:
        return "{:06.4f}    sec".format(no_of_secs)
    elif no_of_secs < 3600:
        return "{:06.4f}    min".format(no_of_secs/60)
    else:
        return "{:06.4f}    hr".format(no_of_secs/3600)

## Load Data

In [4]:
articles_df = pd.read_csv('../data/shared_articles.csv')
articles_df.fillna('UNKOWN', inplace=True)
articles_df.shape

(3122, 13)

In [5]:
articles_df.head()

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
0,1459192779,CONTENT REMOVED,-6451309518266745024,4340306774493623681,8940341205206233829,UNKOWN,UNKOWN,UNKOWN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,UNKOWN,UNKOWN,UNKOWN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,UNKOWN,UNKOWN,UNKOWN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,UNKOWN,UNKOWN,UNKOWN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,UNKOWN,UNKOWN,UNKOWN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en


### Filtering Shared Articles

In [6]:
articles_df['eventType'].value_counts()

CONTENT SHARED     3047
CONTENT REMOVED      75
Name: eventType, dtype: int64

In [7]:
shared_articles_df = articles_df[articles_df['eventType'] == 'CONTENT SHARED']
shared_articles_df.shape

(3047, 13)

### Filtering Shared Articles in English

In [8]:
shared_articles_df['lang'].value_counts()

en    2211
pt     829
la       3
es       2
ja       2
Name: lang, dtype: int64

In [9]:
eng_shared_articles_df = shared_articles_df[shared_articles_df['lang'] == 'en']
eng_shared_articles_df.shape

(2211, 13)

In [10]:
eng_shared_articles_df = eng_shared_articles_df[['contentId', 'url', 'title', 'text']]
eng_shared_articles_df.head()

,contentId,url,title,text
1,-4110354420726924665,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...
2,-7292285110016212249,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...
3,-6151852268067518688,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...
4,2448026894306402386,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...
5,-2826566343807132236,http://www.coindesk.com/ieee-blockchain-oxford...,IEEE to Talk Blockchain at Cloud Computing Oxf...,One of the largest and oldest organizations fo...


In [11]:
len(eng_shared_articles_df['contentId'].unique())

2211

### Combine title and text to create text_content

In [12]:
eng_shared_articles_df['text_content'] = eng_shared_articles_df[['title', 'text']].apply(lambda x: '. '.join(x), axis=1)
eng_shared_articles_df.head()

,contentId,url,title,text,text_content
1,-4110354420726924665,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,"Ethereum, a Virtual Currency, Enables Transact..."
2,-7292285110016212249,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,Bitcoin Future: When GBPcoin of Branson Wins O...
3,-6151852268067518688,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,Google Data Center 360° Tour. We're excited to...
4,2448026894306402386,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,"IBM Wants to ""Evolve the Internet"" With Blockc..."
5,-2826566343807132236,http://www.coindesk.com/ieee-blockchain-oxford...,IEEE to Talk Blockchain at Cloud Computing Oxf...,One of the largest and oldest organizations fo...,IEEE to Talk Blockchain at Cloud Computing Oxf...


In [13]:
eng_shared_articles_df.drop(['title', 'text'], inplace=True, axis=1)
eng_shared_articles_df.head()

,contentId,url,text_content
1,-4110354420726924665,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact..."
2,-7292285110016212249,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...
3,-6151852268067518688,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour. We're excited to...
4,2448026894306402386,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc..."
5,-2826566343807132236,http://www.coindesk.com/ieee-blockchain-oxford...,IEEE to Talk Blockchain at Cloud Computing Oxf...


## Get Unique Text Content

In [14]:
eng_shared_articles_df.head()

,contentId,url,text_content
1,-4110354420726924665,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact..."
2,-7292285110016212249,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...
3,-6151852268067518688,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour. We're excited to...
4,2448026894306402386,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc..."
5,-2826566343807132236,http://www.coindesk.com/ieee-blockchain-oxford...,IEEE to Talk Blockchain at Cloud Computing Oxf...


### Multiple ContentIds, Urls for Text_Content

In [15]:
len(eng_shared_articles_df['contentId'].unique()), len(eng_shared_articles_df['url'].unique()), len(eng_shared_articles_df['text_content'].unique())

(2211, 2191, 2185)

In [16]:
url_contents_df = eng_shared_articles_df[['url', 'text_content']].drop_duplicates()
url_contents_df.shape

(2191, 2)

In [17]:
url_contents_df.head()

,url,text_content
1,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact..."
2,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...
3,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour. We're excited to...
4,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc..."
5,http://www.coindesk.com/ieee-blockchain-oxford...,IEEE to Talk Blockchain at Cloud Computing Oxf...


In [18]:
len(url_contents_df['url'].unique()), len(url_contents_df['text_content'].unique())

(2191, 2185)

In [19]:
len(url_contents_df['url'].unique()) - len(url_contents_df['text_content'].unique())

6

### Multiple Urls for Text Content

In [20]:
content_urls = url_contents_df.groupby('text_content').apply(lambda x : list(x['url'].unique()))
for content in content_urls.index:
    if len(content_urls[content]) > 1:
        #print(content)
        pprint(content_urls[content])

['http://android-developers.blogspot.com.br/2016/04/android-n-developer-preview-2-out-today.html',
 'http://android-developers.blogspot.com/2016/04/android-n-developer-preview-2-out-today.html']
['https://www.oreilly.com/learning/how-to-build-a-robot-that-sees-with-100-and-tensorflow?twitter=@bigdata&utm_source=hackernewsletter&utm_medium=email&utm_term=fav',
 'https://www.oreilly.com/learning/how-to-build-a-robot-that-sees-with-100-and-tensorflow']
['https://www.blog.google/products/g-suite/introducing-google-cloud-search-g-suite/',
 'https://blog.google/products/g-suite/introducing-google-cloud-search-g-suite/']
['https://www.ted.com/talks/linus_torvalds_the_mind_behind_linux',
 'http://www.ted.com/talks/linus_torvalds_the_mind_behind_linux']
['https://www.thinkwithgoogle.com/articles/youtube-empowering-ads-engage.html?utm_source=Gplus&utm_medium=social&utm_campaign=Think',
 'https://www.thinkwithgoogle.com/articles/youtube-empowering-ads-engage.html?utm_medium=email-d&utm_source=201

In [21]:
eng_shared_articles_content_df = eng_shared_articles_df['text_content'].drop_duplicates()\
                                                                       .reset_index()\
                                                                       .rename(columns={'index' : 'text_content_id'})
eng_shared_articles_content_df.shape

(2185, 2)

In [22]:
eng_shared_articles_content_df.head()

,text_content_id,text_content
0,1,"Ethereum, a Virtual Currency, Enables Transact..."
1,2,Bitcoin Future: When GBPcoin of Branson Wins O...
2,3,Google Data Center 360° Tour. We're excited to...
3,4,"IBM Wants to ""Evolve the Internet"" With Blockc..."
4,5,IEEE to Talk Blockchain at Cloud Computing Oxf...


In [23]:
eng_shared_articles_text_id_df = eng_shared_articles_df.merge(eng_shared_articles_content_df)
eng_shared_articles_text_id_df.head()

,contentId,url,text_content,text_content_id
0,-4110354420726924665,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",1
1,-7292285110016212249,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,2
2,-6151852268067518688,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour. We're excited to...,3
3,2448026894306402386,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",4
4,-2826566343807132236,http://www.coindesk.com/ieee-blockchain-oxford...,IEEE to Talk Blockchain at Cloud Computing Oxf...,5


In [24]:
eng_shared_articles_text_id_df['text_content_id'].value_counts()

337     4
80      4
618     3
723     2
838     2
       ..
489     1
491     1
2540    1
493     1
2       1
Name: text_content_id, Length: 2185, dtype: int64

In [25]:
eng_shared_articles_text_id_df[eng_shared_articles_text_id_df['text_content_id'] == 337]

,contentId,url,text_content,text_content_id
280,8160885002406274828,https://medium.com/quality-functions/preventin...,Preventing Software Bugs from Ever Occurring -...,337
281,-4248528062574538011,https://medium.com/quality-functions/preventin...,Preventing Software Bugs from Ever Occurring -...,337
282,-3265730906936163775,https://medium.com/quality-functions/preventin...,Preventing Software Bugs from Ever Occurring -...,337
283,2824996248683640175,https://medium.com/quality-functions/preventin...,Preventing Software Bugs from Ever Occurring -...,337


## Extract Topics of Articles

In [26]:
eng_shared_articles_content_df.shape

(2185, 2)

In [27]:
eng_shared_articles_content_df.head()

,text_content_id,text_content
0,1,"Ethereum, a Virtual Currency, Enables Transact..."
1,2,Bitcoin Future: When GBPcoin of Branson Wins O...
2,3,Google Data Center 360° Tour. We're excited to...
3,4,"IBM Wants to ""Evolve the Internet"" With Blockc..."
4,5,IEEE to Talk Blockchain at Cloud Computing Oxf...


In [28]:
eng_shared_articles_content_df.loc[:, 'processed_text'] = eng_shared_articles_content_df['text_content'].apply(lemmatize_spacy)

In [29]:
eng_shared_articles_content_df.head()

,text_content_id,text_content,processed_text
0,1,"Ethereum, a Virtual Currency, Enables Transact...",Ethereum a Virtual Currency Enables Transactio...
1,2,Bitcoin Future: When GBPcoin of Branson Wins O...,Bitcoin Future when GBPcoin of Branson Wins ov...
2,3,Google Data Center 360° Tour. We're excited to...,Google Data Center Tour We be excite to share ...
3,4,"IBM Wants to ""Evolve the Internet"" With Blockc...",IBM want to evolve the internet with Blockchai...
4,5,IEEE to Talk Blockchain at Cloud Computing Oxf...,ieee to talk Blockchain at Cloud Computing Oxf...


In [30]:
# bow_lda_pipeline = Pipeline(steps=[('vectorizer', CountVectorizer()),
#                                    ('dim_reduce', LatentDirichletAllocation())
#                               ])

# bow_lda_pipeline_params = {
#     'vectorizer__strip_accents': 'unicode',
#     'vectorizer__stop_words': 'english',
#     'vectorizer__lowercase': True,
#     'vectorizer__token_pattern': r'\b[a-zA-Z]{3,}\b',
#     'vectorizer__min_df': 5,
#     'vectorizer__max_df': 0.5,
    
#     'dim_reduce__n_components': 30,
#     'dim_reduce__random_state': 0
# }
# bow_lda_pipeline.set_params(**bow_lda_pipeline_params)

# start_time = default_timer()
# bow_lda_pipeline.fit(eng_shared_articles_content_df['text_content'])
# end_time = default_timer()

# time_taken_str = convert_sec(end_time - start_time)
# print("Time Taken : {}".format(time_taken_str))


# start_time = default_timer()
# pyLDAvis.sklearn.prepare(bow_lda_pipeline.named_steps['dim_reduce'], 
#                          bow_lda_pipeline.named_steps['vectorizer'].fit_transform(eng_shared_articles_content_df['text_content']), 
#                          bow_lda_pipeline.named_steps['vectorizer'])
# end_time = default_timer()

# time_taken_str = convert_sec(end_time - start_time)
# print("Time Taken : {}".format(time_taken_str))

In [31]:
def get_topics(model, feature_names, n_top_words=10):
    topics_df = pd.DataFrame(model.components_, columns=feature_names)   
    topics = dict()
    for i, topic_words in topics_df.iterrows():
        topic_id_str = 'topic_' + str(i)        
        topic_words_dict = topic_words.sort_values(ascending=False).head(n_top_words).to_dict()
        topic_words_str = ', '.join(['#'+str(word) for word in topic_words_dict.keys()])
        print(topic_id_str + " : " + topic_words_str)
        topics[topic_id_str] = dict()
        topics[topic_id_str]['words_str'] = topic_words_str
        topics[topic_id_str]['words'] = topic_words_dict
    return topics

### Term Frequency LDA

In [37]:
tf_vectorizer = CountVectorizer(strip_accents = 'ascii',
                                stop_words = 'english',
                                lowercase = True,
                                analyzer = 'word',
                                token_pattern = r'\b[a-zA-Z]{5,}\b',
                                ngram_range = (1, 1),
                                max_df = 0.8, min_df = 1)
lda_tf = LatentDirichletAllocation(n_components=15, n_jobs=-1, random_state=0)

start_time = default_timer()

dtm_tf = tf_vectorizer.fit_transform(eng_shared_articles_content_df['processed_text'])
lda_tf.fit(dtm_tf)

end_time = default_timer()
time_taken_str = convert_sec(end_time - start_time)
print("Time Taken : {}".format(time_taken_str))

tf_feature_names = tf_vectorizer.get_feature_names()
topics = get_topics(lda_tf, tf_feature_names, n_top_words=10)

# n_components=5
# Time Taken : 10.7162    sec
# topic_0 : #people, #world, #woman, #company, #country, #think, #thing, #million, #space, #write
# topic_1 : #change, #build, #create, #datum, #example, #application, #write, #project, #event, #start
# topic_2 : #google, #cloud, #service, #apple, #platform, #company, #android, #developer, #product, #device
# topic_3 : #company, #customer, #business, #product, #digital, #technology, #drupal, #experience, #datum, #organization
# topic_4 : #machine, #datum, #learn, #learning, #people, #model, #thing, #image, #human, #think

# n_components=10
# Time Taken : 14.4023    sec
# topic_0 : #woman, #people, #country, #world, #music, #black, #space, #brazil, #america, #write
# topic_1 : #software, #change, #project, #build, #process, #development, #design, #organization, #people, #agile
# topic_2 : #google, #apple, #product, #company, #cloud, #platform, #android, #service, #developer, #build
# topic_3 : #drupal, #module, #content, #acquia, #commerce, #feature, #build, #service, #support, #vehicle
# topic_4 : #machine, #learning, #learn, #datum, #people, #human, #computer, #thing, #google, #image
# topic_5 : #company, #customer, #digital, #business, #technology, #datum, #product, #blockchain, #market, #experience
# topic_6 : #datum, #example, #event, #write, #method, #function, #value, #class, #result, #create
# topic_7 : #cloud, #google, #mobile, #docker, #application, #storage, #device, #security, #datum, #password
# topic_8 : #google, #service, #create, #instance, #cloud, #message, #datum, #update, #windows, #application
# topic_9 : #docker, #build, #container, #image, #command, #start, #version, #server, #support, #application
    
# n_components = 15
# Time Taken : 23.2102    sec
# topic_0 : #woman, #country, #brazil, #black, #music, #trend, #quantum, #plague, #death, #america
# topic_1 : #software, #change, #project, #build, #development, #process, #agile, #organization, #service, #developer
# topic_2 : #product, #company, #business, #people, #design, #digital, #service, #build, #platform, #experience
# topic_3 : #drupal, #module, #content, #acquia, #community, #support, #build, #developer, #commerce, #feature
# topic_4 : #machine, #learning, #datum, #learn, #model, #image, #human, #intelligence, #computer, #algorithm
# topic_5 : #customer, #company, #technology, #datum, #digital, #blockchain, #business, #bitcoin, #service, #industry
# topic_6 : #datum, #example, #method, #event, #function, #write, #value, #create, #result, #class
# topic_7 : #password, #token, #kotlin, #field, #label, #place, #swarm, #aggregation, #value, #google
# topic_8 : #google, #android, #facebook, #developer, #search, #windows, #content, #create, #message, #mobile
# topic_9 : #docker, #container, #command, #build, #image, #version, #server, #start, #application, #create
# topic_10 : #apple, #people, #thing, #company, #email, #device, #think, #phone, #feature, #microsoft
# topic_11 : #language, #people, #write, #think, #thing, #story, #change, #learn, #world, #really
# topic_12 : #payment, #account, #store, #customer, #consumer, #online, #credit, #retailer, #wallet, #chart
# topic_13 : #cloud, #google, #service, #application, #platform, #datum, #server, #instance, #support, #network
# topic_14 : #mobile, #people, #experience, #thing, #website, #start, #world, #sound, #content, #think



# n_components = 20
# Time Taken : 32.3148    sec
# topic_0 : #woman, #music, #trend, #quantum, #light, #brazil, #power, #circuit, #gender, #world
# topic_1 : #software, #project, #change, #build, #service, #agile, #microservice, #asset, #component, #development
# topic_2 : #company, #product, #service, #platform, #business, #build, #google, #design, #slack, #developer
# topic_3 : #vehicle, #internet, #technology, #drive, #autonomous, #sensor, #driver, #device, #airbnb, #china
# topic_4 : #machine, #learning, #learn, #human, #google, #intelligence, #computer, #datum, #company, #people
# topic_5 : #company, #technology, #blockchain, #bitcoin, #customer, #financial, #datum, #service, #industry, #business
# topic_6 : #event, #datum, #example, #method, #function, #content, #write, #result, #database, #search
# topic_7 : #google, #field, #design, #label, #place, #aggregation, #apple, #bucket, #swarm, #different
# topic_8 : #google, #facebook, #notification, #message, #search, #android, #drive, #create, #desktop, #share
# topic_9 : #docker, #container, #command, #image, #build, #server, #linux, #windows, #application, #start
# topic_10 : #apple, #email, #people, #thing, #feature, #company, #device, #phone, #microsoft, #iphone
# topic_11 : #people, #thing, #think, #story, #change, #write, #really, #start, #point, #problem
# topic_12 : #consumer, #customer, #brand, #store, #company, #datum, #online, #market, #growth, #percent
# topic_13 : #cloud, #google, #service, #datum, #application, #platform, #instance, #server, #network, #request
# topic_14 : #mobile, #people, #website, #experience, #content, #digital, #account, #world, #thing, #marketing
# topic_15 : #datum, #model, #learn, #language, #machine, #image, #learning, #network, #example, #algorithm
# topic_16 : #password, #event, #salesforce, #trello, #include, #storage, #million, #space, #board, #create
# topic_17 : #customer, #company, #drupal, #business, #product, #organization, #experience, #change, #process, #digital
# topic_18 : #woman, #write, #people, #japanese, #technical, #english, #death, #speak, #black, #release
# topic_19 : #developer, #module, #build, #javascript, #android, #support, #library, #project, #create, #framework

Time Taken : 27.4318    sec
topic_0 : #woman, #country, #brazil, #black, #music, #trend, #quantum, #plague, #death, #america
topic_1 : #software, #change, #project, #build, #development, #process, #agile, #organization, #service, #developer
topic_2 : #product, #company, #business, #people, #design, #digital, #service, #build, #platform, #experience
topic_3 : #drupal, #module, #content, #acquia, #community, #support, #build, #developer, #commerce, #feature
topic_4 : #machine, #learning, #datum, #learn, #model, #image, #human, #intelligence, #computer, #algorithm
topic_5 : #customer, #company, #technology, #datum, #digital, #blockchain, #business, #bitcoin, #service, #industry
topic_6 : #datum, #example, #method, #event, #function, #write, #value, #create, #result, #class
topic_7 : #password, #token, #kotlin, #field, #label, #place, #swarm, #aggregation, #value, #google
topic_8 : #google, #android, #facebook, #developer, #search, #windows, #content, #create, #message, #mobile
topic_9 : #

In [38]:
dtm_tf.shape

(2185, 29751)

In [39]:
lda_tf.components_.shape

(15, 29751)

In [40]:
reduced_dtm_tf = pd.DataFrame(lda_tf.transform(dtm_tf), columns=['topic_' + str(i) for i in range(lda_tf.n_components)])
reduced_dtm_tf.shape

(2185, 15)

In [41]:
reduced_dtm_tf.head()

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14
0,0.000168,0.081273,0.210969,0.000168,0.086958,0.618952,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168
1,0.000450,0.000450,0.000450,0.000450,0.122925,0.085803,0.000450,0.000450,0.000450,0.000450,0.000450,0.000450,0.682560,0.000450,0.103758
2,0.000680,0.000680,0.000680,0.000680,0.000680,0.000680,0.000680,0.000680,0.604374,0.000680,0.131411,0.000680,0.000680,0.256052,0.000680
3,0.000151,0.031980,0.000151,0.000151,0.000151,0.551659,0.000151,0.000151,0.023101,0.000151,0.000151,0.000151,0.000151,0.000151,0.391597
4,0.123703,0.000538,0.000538,0.000538,0.146711,0.500976,0.000538,0.038940,0.000538,0.000538,0.000538,0.000538,0.000538,0.184294,0.000538


In [42]:
def get_topic_names(x):
    return x[x>0.1].to_dict()
reduced_dtm_tf.loc[:, 'mostly_about'] = reduced_dtm_tf.apply(get_topic_names, axis=1)

In [43]:
reduced_dtm_tf.head()

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,mostly_about
0,0.000168,0.081273,0.210969,0.000168,0.086958,0.618952,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,"{'topic_2': 0.2109692974037969, 'topic_5': 0.6..."
1,0.000450,0.000450,0.000450,0.000450,0.122925,0.085803,0.000450,0.000450,0.000450,0.000450,0.000450,0.000450,0.682560,0.000450,0.103758,"{'topic_4': 0.12292520324381467, 'topic_12': 0..."
2,0.000680,0.000680,0.000680,0.000680,0.000680,0.000680,0.000680,0.000680,0.604374,0.000680,0.131411,0.000680,0.000680,0.256052,0.000680,"{'topic_8': 0.6043744437668673, 'topic_10': 0...."
3,0.000151,0.031980,0.000151,0.000151,0.000151,0.551659,0.000151,0.000151,0.023101,0.000151,0.000151,0.000151,0.000151,0.000151,0.391597,"{'topic_5': 0.551659427679873, 'topic_14': 0.3..."
4,0.123703,0.000538,0.000538,0.000538,0.146711,0.500976,0.000538,0.038940,0.000538,0.000538,0.000538,0.000538,0.000538,0.184294,0.000538,"{'topic_0': 0.12370324630402275, 'topic_4': 0...."


In [44]:
reduced_dtm_tf.loc[:, 'no_of_topics'] = reduced_dtm_tf['mostly_about'].apply(lambda x: len(x))

In [45]:
reduced_dtm_tf.head()

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,mostly_about,no_of_topics
0,0.000168,0.081273,0.210969,0.000168,0.086958,0.618952,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,"{'topic_2': 0.2109692974037969, 'topic_5': 0.6...",2
1,0.000450,0.000450,0.000450,0.000450,0.122925,0.085803,0.000450,0.000450,0.000450,0.000450,0.000450,0.000450,0.682560,0.000450,0.103758,"{'topic_4': 0.12292520324381467, 'topic_12': 0...",3
2,0.000680,0.000680,0.000680,0.000680,0.000680,0.000680,0.000680,0.000680,0.604374,0.000680,0.131411,0.000680,0.000680,0.256052,0.000680,"{'topic_8': 0.6043744437668673, 'topic_10': 0....",3
3,0.000151,0.031980,0.000151,0.000151,0.000151,0.551659,0.000151,0.000151,0.023101,0.000151,0.000151,0.000151,0.000151,0.000151,0.391597,"{'topic_5': 0.551659427679873, 'topic_14': 0.3...",2
4,0.123703,0.000538,0.000538,0.000538,0.146711,0.500976,0.000538,0.038940,0.000538,0.000538,0.000538,0.000538,0.000538,0.184294,0.000538,"{'topic_0': 0.12370324630402275, 'topic_4': 0....",4


In [46]:
reduced_dtm_tf['no_of_topics'].value_counts()

2    825
3    790
4    263
1    255
5     51
6      1
Name: no_of_topics, dtype: int64

In [47]:
reduced_dtm_tf[reduced_dtm_tf['no_of_topics'] == 0]

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,mostly_about,no_of_topics


In [48]:
topics

{'topic_0': {'words_str': '#woman, #country, #brazil, #black, #music, #trend, #quantum, #plague, #death, #america',
  'words': {'woman': 289.0235386792721,
   'country': 158.21889710255434,
   'brazil': 137.60603062522927,
   'black': 119.91825424776653,
   'music': 95.7943771986731,
   'trend': 81.39803435926935,
   'quantum': 77.23813796587133,
   'plague': 69.69554502290272,
   'death': 66.31711949439827,
   'america': 59.02767034060135}},
 'topic_1': {'words_str': '#software, #change, #project, #build, #development, #process, #agile, #organization, #service, #developer',
  'words': {'software': 969.4987918685139,
   'change': 703.2200174689532,
   'project': 670.8877562289442,
   'build': 566.5540015018217,
   'development': 564.0065049722285,
   'process': 498.4440280422339,
   'agile': 485.9623118911154,
   'organization': 404.1648888063069,
   'service': 347.3021201178763,
   'developer': 343.9782715653292}},
 'topic_2': {'words_str': '#product, #company, #business, #people, #de

In [49]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

/home/ravi/recommendation_system/recommender_system/rec_sys_env/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.144774 -0.050035       1        1  13.525094
4     -0.004657  0.014986       2        1  12.828754
5      0.066064  0.183712       3        1  11.886457
2      0.042413  0.144496       4        1  10.830029
13    -0.173971  0.062413       5        1   9.398981
10     0.050079  0.000165       6        1   8.097757
1     -0.073929  0.066920       7        1   6.971852
8     -0.073956 -0.035232       8        1   5.915535
14     0.043999 -0.035346       9        1   5.122229
11     0.083070 -0.078391      10        1   3.875720
9     -0.097405 -0.116462      11        1   3.233102
3     -0.046987  0.036495      12        1   2.566767
12     0.141688  0.060895      13        1   1.993615
7     -0.051172 -0.132614      14        1   1.946124
0      0.239539 -0.122002      15        1   1.807984, topic_info=      Category         Freq      Term        Total  loglift  logprob
10871  Default  5174.000000    google  5174.000000  30.0000  30.0000
4677   Default  2369.000000     cloud  2369.000000  29.0000  29.0000
7841   Default  1045.000000    drupal  1045.000000  28.0000  28.0000
1201   Default  1656.000000     apple  1656.000000  27.0000  27.0000
15458  Default  2694.000000   machine  2694.000000  26.0000  26.0000
...        ...          ...       ...          ...      ...      ...
8451   Topic15    43.250721  engineer   730.262735   1.1866  -5.7850
29410  Topic15    48.442912     world  1946.114772   0.3198  -5.6716
24969  Topic15    40.987105     state   736.042984   1.1249  -5.8387
19328  Topic15    44.892728    people  3528.860114  -0.3515  -5.7477
16515  Topic15    41.294317   million  1031.095962   0.7953  -5.8313

[1189 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
122        1  0.120003      access
122        2  0.050770      access
122        3  0.141234      access
122        4  0.079386      access
122        5  0.313853      access
...      ...       ...         ...
29705      6  0.916526      zigbee
29733      4  0.152851   zookeeper
29733      9  0.764257   zookeeper
29733     13  0.038213   zookeeper
29745     11  0.873597  zumbrunnen

[5107 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 5, 6, 3, 14, 11, 2, 9, 15, 12, 10, 4, 13, 8, 1])

#### Using different MDS functions

With `sklearn` installed, other MDS functions, such as MMDS and TSNE can be used for plotting if the default PCoA is not satisfactory.

In [50]:
# pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='mmds')

In [51]:
# pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='tsne')

### Tf-IDf LDA

In [52]:
tf_idf_vectorizer = TfidfVectorizer(strip_accents = 'ascii',
                                    stop_words = 'english',
                                    lowercase = True,
                                    analyzer = 'word',
                                    token_pattern = r'\b[a-zA-Z]{5,}\b',
                                    ngram_range = (1, 1),
                                    max_df = 0.8, min_df = 1,
                                    #max_features = 1000,
                                    norm='l2', 
                                    use_idf=True, smooth_idf=True, sublinear_tf=True)
lda_tf_idf = LatentDirichletAllocation(n_components=15, random_state=0)

start_time = default_timer()

dtm_tf_idf = tf_idf_vectorizer.fit_transform(eng_shared_articles_content_df['text_content'])
lda_tf_idf.fit(dtm_tf_idf)

end_time = default_timer()
time_taken_str = convert_sec(end_time - start_time)
print("Time Taken : {}".format(time_taken_str))

tf_idf_feature_names = tf_idf_vectorizer.get_feature_names()
topics = get_topics(lda_tf_idf, tf_idf_feature_names, n_top_words=10)

Time Taken : 13.7012    sec
topic_0 : #kittlaus, #bryant, #bluemix, #shots, #whiteboard, #jamboard, #sussman, #instrumentation, #disclosure, #bootloader
topic_1 : #firebase, #swarm, #balancer, #microservice, #stackdriver, #downtime, #gcloud, #layout, #niantic, #proxy
topic_2 : #aggregations, #aggregation, #elasticsearch, #facets, #raspberry, #sorted, #buffer, #gradle, #macos, #sierra
topic_3 : #mycroft, #thunder, #burda, #kickstarter, #tesla, #skully, #walgreens, #publishing, #interviewer, #coupons
topic_4 : #layout, #arguments, #debate, #bigquery, #graphs, #constraints, #angular, #optimization, #micro, #checklist
topic_5 : #sprints, #kotlin, #optionals, #dagger, #kibana, #espresso, #retrolambda, #subcommittee, #unicode, #safari
topic_6 : #female, #scandic, #portrait, #kotlin, #bigquery, #sarcastic, #istanbul, #tattoo, #vintage, #dzone
topic_7 : #acquia, #protocol, #whatsapp, #ubuntu, #dynamic, #keyboard, #caching, #optimization, #exposed, #dropbox
topic_8 : #evaluations, #martech, #br

In [53]:
dtm_tf_idf.shape

(2185, 36915)

In [54]:
lda_tf_idf.components_.shape

(15, 36915)

In [55]:
reduced_dtm_tf_idf = lda_tf_idf.transform(dtm_tf_idf)
reduced_dtm_tf_idf.shape

(2185, 15)

In [56]:
pyLDAvis.sklearn.prepare(lda_tf_idf, dtm_tf_idf, tf_idf_vectorizer)

/home/ravi/recommendation_system/recommender_system/rec_sys_env/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
11     0.323602  0.003820       1        1  73.713030
7     -0.052688  0.061933       2        1   4.385899
4     -0.047838 -0.024758       3        1   3.848177
1     -0.028561 -0.005556       4        1   2.162808
2     -0.023090 -0.007266       5        1   1.812133
8     -0.020890 -0.004497       6        1   1.656522
9     -0.020051 -0.004711       7        1   1.608608
10    -0.018986 -0.004252       8        1   1.529938
0     -0.017311 -0.003303       9        1   1.421312
5     -0.017154 -0.002765      10        1   1.415194
12    -0.017143 -0.002897      11        1   1.412310
14    -0.016569 -0.002712      12        1   1.371388
6     -0.016289 -0.002226      13        1   1.340625
3     -0.014161 -0.000821      14        1   1.204943
13    -0.012870  0.000009      15        1   1.117115, topic_info=      Category      Freq        Term     Total  loglift  logprob
276    Default  2.000000      acquia  2.000000  30.0000  30.0000
18313  Default  1.000000      layout  1.000000  29.0000  29.0000
25575  Default  1.000000    protocol  1.000000  28.0000  28.0000
10026  Default  1.000000     dynamic  1.000000  27.0000  27.0000
32599  Default  2.000000  techcrunch  2.000000  26.0000  26.0000
...        ...       ...         ...       ...      ...      ...
29634  Topic15  0.043689    sidewalk  0.312141   2.5281  -8.8586
11408  Topic15  0.045495  expiration  0.394162   2.3353  -8.8181
24531  Topic15  0.048053     pokemon  0.546222   2.0637  -8.7634
33550  Topic15  0.045769      trades  0.471860   2.1614  -8.8121
16920  Topic15  0.043734     invites  0.450633   2.1619  -8.8576

[672 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
276        2  0.950066     acquia
1228       3  1.022156    angular
1655       3  0.942474  arguments
2670       1  0.998782      based
3033       1  0.989007     better
...      ...       ...        ...
35037      1  0.990168      using
35590      2  1.262473     vmware
36119      2  0.716697   whatsapp
36506      1  0.978702      world
36520      2  0.644732      worry

[96 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 8, 5, 2, 3, 9, 10, 11, 1, 6, 13, 15, 7, 4, 14])

#### Using different MDS functions

With `sklearn` installed, other MDS functions, such as MMDS and TSNE can be used for plotting if the default PCoA is not satisfactory.

In [57]:
# pyLDAvis.sklearn.prepare(lda_tf_idf, dtm_tf_idf, tf_idf_vectorizer, mds='mmds')

In [58]:
# pyLDAvis.sklearn.prepare(lda_tf_idf, dtm_tf_idf, tf_idf_vectorizer, mds='tsne')